Подключаем необходимые библиотеки

In [1]:
import numpy as np
import re
import pandas as pd
import string
from collections import defaultdict
from sklearn import metrics
from time import time
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
import re
import string
from collections import defaultdict
from sklearn import metrics
from time import time
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
import pandas as jtp
from nltk.corpus import wordnet
from sklearn.metrics.cluster import homogeneity_score
from sklearn.metrics.cluster import completeness_score 
from sklearn.metrics.cluster import v_measure_score

Подгружаем датасет labeled.csv

In [2]:
path = '/home/roman/ds_bootcamp/ds-phase-2/toxic_comments_detection/labeled.csv'
df = pd.read_csv(path)

In [3]:
df

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


In [4]:
labels = df.comment
unique_labels, category_sizes = np.unique(labels, return_counts=True)
true_k = unique_labels.shape[0]

In [11]:
evaluations = []
evaluations_std = []


### функция проверки 

def fit_and_evaluate(km, X, name=None, n_runs=5):
    name = km.__class__.__name__ if name is None else name

    train_times = []
    scores = defaultdict(list)
    for seed in range(n_runs):
        km.set_params(random_state=seed)
        t0 = time()
        km.fit(X)
        train_times.append(time() - t0)
        scores["Homogeneity"].append(metrics.homogeneity_score(labels, km.labels_))
        scores["Completeness"].append(metrics.completeness_score(labels, km.labels_))
        scores["V-measure"].append(metrics.v_measure_score(labels, km.labels_))
        scores["Adjusted Rand-Index"].append(
            metrics.adjusted_rand_score(labels, km.labels_)
        )
        scores["Silhouette Coefficient"].append(
            metrics.silhouette_score(X, km.labels_, sample_size=2000)
        )
    train_times = np.asarray(train_times)

    print(f"clustering done in {train_times.mean():.2f} ± {train_times.std():.2f} s ")
    evaluation = {
        "estimator": name,
        "train_time": train_times.mean(),
    }
    evaluation_std = {
        "estimator": name,
        "train_time": train_times.std(),
    }
    for score_name, score_values in scores.items():
        mean_score, std_score = np.mean(score_values), np.std(score_values)
        print(f"{score_name}: {mean_score:.3f} ± {std_score:.3f}")
        evaluation[score_name] = mean_score
        evaluation_std[score_name] = std_score
    evaluations.append(evaluation)
    evaluations_std.append(evaluation_std)

Чистим датасет от мусорных символов, которые не несут смысловую нагрузку для дальнейшей кластеризации

In [39]:
def clean(text):
    text = text.lower()
    text = re.sub(r'https\S+', " ", text)
    text = re.sub(r'@\w+',' ', text)
    text = re.sub(r'#w+', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'<.*?>', ' ', text)
    text = re.sub('[/+_!@#$A-Za-z0-9\n.,:()""«»;-]', ' ', text)
    return text

cleaned_text = []

for text in df.comment:
    cleaned_text.append(clean(text))

Проводим лемматизацию

In [40]:
wn_lemmatizer = WordNetLemmatizer()

lemmatized_text = []
for tweet in cleaned_text:
    lemmatized_text.append(' '.join([wn_lemmatizer.lemmatize(word) for word in tweet.split()]))

In [41]:
lemmatized_text

['верблюдовто за что дебилы бл',
 'хохлы это отдушина затюканого россиянина мол вон а у хохлов еще хуже если бы хохлов не было кисель их бы придумал',
 'собаке собачья смерть',
 'страницу обнови дебил это тоже не оскорбление а доказанный факт недебил про себя во множественном числе писать не будет или мы в тебя верим это ты и твои воображаемые друзья',
 'тебя не убедил страничный пдф в том что скрипалей отравила россия анализировать и думать пытаешься ватник что ли',
 'для каких стан является эталоном современная система здравоохранения рф для зимбабве ты тупой хохлы',
 'в шапке были ссылки на инфу по текущему фильму марвел эти ссылки были заменены на фразу репортим брипидора игнорируем его посты если этого недостаточно чтобы понять что модератор абсолютный неадекват и его нужно лишить полномочий тогда эта борда пробивает абсолютное дно по неадекватности',
 'упад т там нельзя строить технологий нет разворуют как всегда уже трещинами пош л тупые китазы не могут ничего нормально сделать'

Проведение токенизации документов

In [42]:
reg_tokenizer = RegexpTokenizer('\w+')
tokenized_text = reg_tokenizer.tokenize_sents(lemmatized_text)

Удаление стоп-слов

In [43]:
sw = stopwords.words('russian')
clean_tokenized_text = []
for i, element in enumerate(tokenized_text):
    clean_tokenized_text.append(' '.join([word for word in element if word not in sw]))

In [44]:
clean_tokenized_text

['верблюдовто дебилы бл',
 'хохлы это отдушина затюканого россиянина мол вон хохлов хуже хохлов кисель придумал',
 'собаке собачья смерть',
 'страницу обнови дебил это оскорбление доказанный факт недебил множественном числе писать верим это твои воображаемые друзья',
 'убедил страничный пдф скрипалей отравила россия анализировать думать пытаешься ватник',
 'каких стан является эталоном современная система здравоохранения рф зимбабве тупой хохлы',
 'шапке ссылки инфу текущему фильму марвел ссылки заменены фразу репортим брипидора игнорируем посты недостаточно понять модератор абсолютный неадекват нужно лишить полномочий эта борда пробивает абсолютное дно неадекватности',
 'упад т строить технологий разворуют трещинами пош л тупые китазы могут нормально сделать',
 'ебать разносит шизик',
 'обосрался сиди обтекай',
 'пишешь хуйню дегенерат поцелуй губы поцелую засос',
 'бактерия тебе понять ист е рик поподробнее сахаров ан вещает какойто бред написал всё логично фраза обращена тебе твои п

Строим векторное представление сообщений с помощью Tf-IDF

In [26]:
vectorizer = TfidfVectorizer(
    max_df=0.5,
    min_df=5,
)

t0 = time()
X_tfidf = vectorizer.fit_transform(clean_tokenized_text)
print(f'Vectorization done in {time() - t0:.3f} s.')
print(f'n_samples: {X_tfidf.shape[0]}, n_features: {X_tfidf.shape[1]}')

Vectorization done in 0.428 s.
n_samples: 14412, n_features: 8070


In [ ]:
# for seed in range(5):
#     kmeans = KMeans(
#         n_clusters=2,
#         max_iter=100,
#         n_init=1, random_state=seed,
#     ).fit(X_tfidf)
#     cluster_ids, cluster_sizes = np.unique(kmeans.labels_, return_counts=True)
#     print(f"Number of elements asigned to each cluster: {cluster_sizes}")
# print(
#     "True number of documents in each category according to the class labels: "
#     f"{category_sizes}")

In [ ]:
# kmeans = KMeans(
#     n_clusters=2,
#     max_iter=100,
#     n_init=5,
# )
# fit_and_evaluate(kmeans, X_tfidf, name='KMeans\non tf-idf vectors')